# Running MMS-LID inference in Colab

## Step 1: Clone fairseq-py and install latest version

In [ ]:
import os

!git clone https://github.com/pytorch/fairseq

# Change current working directory
!pwd
%cd "/content/fairseq"
!pip install --editable ./
!pip install tensorboardX


## 2. Download MMS-LID model



In [2]:
available_models = ["l126", "l256", "l512", "l1024", "l2048", "l4017"]

# We will use L126 model which can recognize 126 languages
model_name = available_models[0] # l126
print(f"Using model - {model_name}")
print(f"Visit https://dl.fbaipublicfiles.com/mms/lid/mms1b_{model_name}_langs.html to check all the languages supported by this model.")

! mkdir -p /content/models_lid
!wget -P /content/models_lid/{model_name} 'https://dl.fbaipublicfiles.com/mms/lid/mms1b_{model_name}.pt'
!wget -P /content/models_lid/{model_name} 'https://dl.fbaipublicfiles.com/mms/lid/dict/l126/dict.lang.txt'



Using model - l126
Visit https://dl.fbaipublicfiles.com/mms/lid/mms1b_l126_langs.html to check all the languages supported by this model.
--2024-12-25 02:01:32--  https://dl.fbaipublicfiles.com/mms/lid/mms1b_l126.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.10, 13.227.219.70, 13.227.219.59, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3856229421 (3.6G) [binary/octet-stream]
Saving to: ‘/content/models_lid/l126/mms1b_l126.pt’

mms1b_l126.pt       100%[===================>]   3.59G  21.2MB/s    in 2m 27s  

2024-12-25 02:04:00 (25.0 MB/s) - ‘/content/models_lid/l126/mms1b_l126.pt’ saved [3856229421/3856229421]

--2024-12-25 02:04:00--  https://dl.fbaipublicfiles.com/mms/lid/dict/l126/dict.lang.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.59, 13.227.219.70, 13.227.219.10, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaip

## 3. Prepare manifest files
Create a folder on path '/content/audio_samples/' and upload your .wav audio files that you need to recognize e.g. '/content/audio_samples/abc.wav' , '/content/audio_samples/def.wav' etc...

Note: You need to make sure that the audio data you are using has a sample rate of 16kHz You can easily do this with FFMPEG like the example below that converts .mp3 file to .flac and fixing the audio sample rate

Here, we use three examples - one audio file from English, Hindi, Chinese each.

In [4]:
! mkdir -p /content/audio_samples/
for key in ["en_us", "hi_in", "cmn_hans_cn"]:
  !wget -O /content/audio_samples/tmp.mp3 https://datasets-server.huggingface.co/assets/google/fleurs/--/{key}/train/0/audio/audio.mp3
  !ffmpeg -hide_banner -loglevel error -y -i   /content/audio_samples/tmp.mp3 -ar 16000 /content/audio_samples/{key}.wav

! mkdir -p /content/audio_samples/


--2024-12-25 02:04:50--  https://datasets-server.huggingface.co/assets/google/fleurs/--/en_us/train/0/audio/audio.mp3
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 18.239.94.9, 18.239.94.106, 18.239.94.90, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|18.239.94.9|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-12-25 02:04:50 ERROR 403: Forbidden.

[mp3 @ 0x555d7e9b4040] Failed to read frame size: Could not seek to 1026.
/content/audio_samples/tmp.mp3: Invalid argument
--2024-12-25 02:04:50--  https://datasets-server.huggingface.co/assets/google/fleurs/--/hi_in/train/0/audio/audio.mp3
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 18.239.94.9, 18.239.94.106, 18.239.94.90, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|18.239.94.9|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-12-25 02:04:50 ERROR 

In [ ]:
! mkdir -p /content/manifest/
import os
with open("/content/manifest/dev.tsv", "w") as ftsv, open("/content/manifest/dev.lang", "w") as flang:
  ftsv.write("/\n")

  for fl in os.listdir("/content/audio_samples/"):
    if not fl.endswith(".wav"):
      continue
    audio_path = f"/content/audio_samples/{fl}"
    # duration should be number of samples in audio. For inference, using a random value should be fine.
    duration = 1234
    ftsv.write(f"{audio_path}\t{duration}\n")
    flang.write("eng\n") # This is the "true" language for the audio. For inference, using a random value should be fine.


# 4: Run Inference and transcribe your audio(s)


In [ ]:
import os

os.environ["PYTHONPATH"] = "/content/fairseq"
os.environ["PREFIX"] = "INFER"
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["USER"] = "mms_lid_user"

!python3 examples/mms/lid/infer.py /content/models_lid/{model_name} --path /content/models_lid/{model_name}/mms1b_l126.pt \
  --task audio_classification  --infer-manifest /content/manifest/dev.tsv --output-path /content/manifest/

2023-05-25 18:19:19.545731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 18:19:21.567795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
| loading model from /content/models_lid/l126/mms1b_l126.pt
2023-05-25 18:19:29 | INFO | fairseq.tasks.audio_classification | Using dict_path : /content/models_lid/l126/dict.lang.txt
2023-05-25 18:19:29 | INFO | root | === Number of labels = 126
2023-05-25 18:20:01 | INFO | fairseq.data.audio.raw_audio_dataset | loaded 3, skipped 0 samples
2023-05-25 18:20:01 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-05-25 18:20:01 | INFO | fairseq.tasks.fairseq_task | reuse_dataloader = True
2023-05-25 18:20:01 | INFO | fairseq.tasks.fair

In [ ]:
print("----- INPUT FILES -----")
! tail -n +2 /content/manifest/dev.tsv

print("\n----- TOP-K PREDICTONS WITH SCORE -----")
! cat /content/manifest//predictions.txt

----- INPUT FILES -----
/content/audio_samples/hi_in.wav	1234
/content/audio_samples/en_us.wav	1234
/content/audio_samples/cmn_hans_cn.wav	1234

----- TOP-K PREDICTONS WITH SCORE -----
[["hin", 0.9931250810623169], ["urd", 0.005808886140584946], ["snd", 0.0005312535213306546]]
[["eng", 0.9989539980888367], ["fas", 0.00036296260077506304], ["haw", 7.031611312413588e-05]]
[["cmn", 0.9996059536933899], ["bod", 0.0002111078501911834], ["kor", 9.211552242049947e-05]]
